In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Weight Capacity'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=2290

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]   

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    tuples=tuple(split_digits(s, *args, **kwargs))
    print(tuples)
    return tuples 

In [20]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

5849

In [21]:
regex_pattern_multi = r"(?i)(supports\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|(\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|()"
regex_pattern_regular = r"(?i)(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?weight capacity)|()"
regex_pattern_na = r"(?i)(\d+\s?lb)|weigh|holds|capacity|()"
regex_pattern_max = r"(?i)(\s?\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|()"               

In [ ]:
df['max_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_max, x))
df['na_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df['multi_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_multi, x))
df['regular'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_regular, x))

In [ ]:
df_max=df[df['max_matches'].astype(str)!='[]']
print(len(df_max))
multi=df[df['multi_matches'].astype(str)!='[]']
print(len(multi))

# Na

In [ ]:
df_na=df[df['max_matches'].astype(str)=='[]']
df_na=df_na[df_na['multi_matches'].astype(str)=='[]']
df_na=df_na[df_na['regular'].astype(str)=='[]']

In [ ]:
match_na=df_na
match_na[curation_col]=r'n/a'
# match_na.head()

# Regular

In [ ]:
df_regular=df[df['regular'].astype(str)!='[]']
print(len(df_regular))
regex_pattern_final_regular = r"(?i)(\d+)|()"
df_regular['final_regular'] = df_regular['regular'].apply(lambda x: re_extract(regex_pattern_final_regular, str(x)))
# df_regular['final_regular']=df_regular['final_regular'].apply(lambda x: str(x).lower())
# # df_regular['final_regular']= df_regular['final_regular'].apply(lambda x: sorted(x, key=natural_key))
# df_regular['final_regular'].apply(lambda x: sorted(x, key=natural_key))
df_regular['final_regular']=df_regular['final_regular'].apply(lambda x:str(x)+' lb').apply(lambda x:re.sub('3000','3,000',str(x))).apply(lambda x:re.sub('1350','1,350',str(x)))

In [ ]:
df_regular[0:500]

# Max

In [36]:
df_max['final_max_matches']=df_max['max_matches'].apply(lambda x: re_extract(regex_pattern_final_regular, str(x)))
print(len(df_max))
regex_pattern_comma=r','
df_max['comma_matches'] = df_max['final_max_matches'].apply(lambda x: re_extract(regex_pattern_comma, str(x)))
df_max_final=df_max[df_max['comma_matches'].astype(str)!='[]']
print(len(df_max_final))
df_max_final['final_max_matches'].explode().value_counts()
df_max_final

1395
48


product_id external_id  \
1      817477818    62764636   
81     817220930    43098996   
122   1239494489    69715815   
320   1341415681    69761326   
380    817279671    63372557   
472   1144336390    69229282   
774    994509696    69327391   
1115   817513832    47410640   
1574   988564251    69598206   
1772   817279672    63372564   
1808  1340348879    69761041   
1889   817628563    69475750   
2053  1067837606    69683144   
2365   817284717    63699883   
2375  1341415691    69761039   
2417   988489079    69629202   
2784   817562304    65326770   
3004  1347890669    66776871   
3035   817311204    68364991   
3263   817540207    63372540   
3327   817625034    69327407   
3468  1124486853    69686215   
3520   988690431    67514717   
3527  1336456002    69589276   
3562   998993423    69640351   
3685   988381856    65355794   
3748  1239494490    69715814   
3779  1067837608    69683143   
3786  1144336389    69229299   
3849  1124691246    69587658   
3927  1158247121    69542400   
3929   988558824    69475749   
3963   817625143    69327384   
4262   988489078    69628114   
4299   817300323    64656182   
4385   817560791    65166291   
4499   988726094    68675639   
4915   998993422    69640354   
5044   817268842    44508029   
5076   998993420    69640357   
5085  1341415690    69761038   
5224   817405621    69475748   
5368  1004989481    63881639   
5445   988489077    69629205   
5510   817402151    69364624   
5541   988489076    69628112   
5814  1067837607    69683145   
5817   817628562    69475747   

                                                                              product_name  \
1                                               Southern Enterprises Niles Corner TV Stand   
81                                                 Trixie Small Soft Sided Mobile Play Pen   
122                                    Babyletto Gelato Portable Bassinet in White/Natural   
320                                       Safavieh Izzy Rectangular Counter Table in White   
380                                        BABYBJÖRN® Fabric Seat for Bouncer in Sand Grey   
472                                          Graco® Pack N' Play® FoldLite Playard in Remi   
774                        Graco® Pack 'n Play Playard with Travel Dome Bassinet in Archie   
1115                                             Safavieh Easton Club Chair in Taupe/White   
1574                                     Dream on Me Palm 3-in-1 Bassinet Playpen in White   
1772                                BABYBJÖRN® Fabric Seat for Bouncer in Anthracite Black   
1808                                                      Baxton Studio Arran Sofa in Grey   
1889                                    WonderFold Wagon X2 Double Stroller Wagon in Black   
2053                                              DaVinci Archie Portable Bassinet in Grey   
2365                                Winsome Pullman 5-Piece Extension Dining Set in Walnut   
2375                                           Baxton Studio Mona Faux Leather Sofa in Tan   
2417                                                 Cybex Gazelle S Stroller in Soho Grey   
2784               Forest Gate Arvada 3-Piece Acacia Wood Outdoor Picnic Set in Dark Brown   
3004                                 Never Rust Outdoor Aluminum Sling Kids Chair in Brown   
3035                         Forest Gate Arvada 3-Piece Acacia Wood Patio Conversation Set   
3263                                   BABYBJÖRN® Fabric Seat for Bouncer in Midnight Blue   
3327                                Graco® Pack N' Play® Travel Dome DLX Playard in Archer   
3468                            BÉABA® by Shnuggle Air Complete Sleep System in Grey/White   
3520              At Water Living Dana Full Faux Leather Upholstered Platform Bed in White   
3527                               Crosley Capella 2-Piece Wicker Outdoor Sofa Set in Grey   
3562                               Silver Cross Wave 2021 Convertible Stroller in Charc

In [21]:
df_max.head()

product_id external_id  \
1    817477818    62764636   
2    817186606    62160568   
3   1348011402    62091169   
13  1345537616    16111970   
14  1339037348    69752483   

                                                      product_name  \
1                       Southern Enterprises Niles Corner TV Stand   
2                         Madison Park Heston Arm Chair in Natural   
3   Margaritaville® \"Relax\" Bar Stools in Grey/Yellow (Set of 2)   
13                        Storage Ottoman with Two Bent-wood Trays   
14         Glitzhome® Upholstered Dining Chairs in Grey (Set of 2)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
1                                                                                                                                       The simple, minimalist look of the Niles Corner TV Stand from Southern Enterprises is the perfect piece to add your family room. The versatile stand showcases a triangular design, 3 functional tiers and accommodates a flat screen TV up to 32.5\" W. Matte black metal creates strong, clean lines surrounding the sizeable glass shelves Triangular design fits this corner TV stand into tight spaces Accommodates a flat screen TV up to 32.5\" W 2 open glass shelves and spacious glass top Multifunctional design fits corner or flat wall Matte black finish Powder-coated iron/5mm tempered, beveled glass Wipe clean Assembly required Measures 34.5\" W x 24.25\" D x 23.25\" H Weighs 52 lb. 85 lb. weight capacity (top)/20 lb. weight capacity (each shelf) 1-year limited manufacturer's warranty Imported   
2                                                                                                                                                                                                                                                                                                                                                        The tall, sleek silhouette of the Heston Arm Chair from Madison Park is accentuated with silver nailhead trim. This elegant and comfy chair features hardwood legs with a Moroccan finish, curved arms, quilted fabric and a high back. Adds elegant style to your home décor High back design Curved arms Nailhead trim with a silver finish Curved solid wood legs with a Moroccan finish Solid wood/foam/polyester/metal 100% foam fill Spot clean Assembly required Measures 27.76\" D x 32.28\" W x 41.33\" H 20.07\" seat depth 19.88\" seat height Weighs 30.65 lb. 300 lb. weight capacity Imported   
3                                                                                                                                                                                                                                                                                                                                                                                                                   Entertain your guests in style with the fun \"Relax\" Bar Stools from Margaritaville. In bold yellow and grey tones, these stools are crafted of wood and feature a weather and water resistan

In [30]:
x=[20, 85]
np.max(x)

85

In [31]:
import numpy as np
df_max['Max'] = df_max['final_max_matches'].apply(lambda x: np.max(x))
# df_max['Min'] = df_max[['length_new', 'width_new']].values.min(1)
df_max

TypeError: cannot perform reduce with flexible type

In [19]:
# Replacement
df_max=df_max[df_max['max_matches'].astype(str)!='[]']
df_max['max_matches']= df_max['max_matches'].apply(lambda x: [replace_all(string, replacement_max) for string in x] if type(x)==list else x)

NameError: name 'replacement_max' is not defined

In [85]:
# Replacement
df_max['max_matches']= df_max['max_matches'].apply(
    lambda x: [replace_all(string, replacement_max) for string in x] if type(x)==list else x)
df_max.head(1)

product_id external_id  \
2   988798592    62739252   

                                                           product_name  \
2  Modway Fortuna Outdoor 5-Piece Patio Conversation Set in Mocha/Brown   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     long_desc  \
2  From the Fortuna Outdoor Series, the 5-Piece Patio Conversation Set by Modway has everything you need to create a great outdoor environment. With sleek, modern design and all-weather cushions, and water- and UV-resistant powder-coated aluminum frames. Powder-coated aluminum frame Modern mid-century look All-weather cushions Removable all-weather cushions Black plastic base glides 350 lb. weight capacity Set includes: 2 arm chairs, measure 30.5\" L x 30\" W x 32.5\" H each 2 ottomans, measure 30.5\" L x 29.5\" W x 16.5\" H each 1 side table, measures 29.5\" L x 29.5\" W x 16.5\" H Powder-coated aluminum frame/polyester cushion/tempered glass Machine washable cushion Assembly required 1-year manufacturer's warranty Imported   

   customer_id attributes                  buckets  bucket_id values  \
2            5       None  Furniture Variety Packs       7491   None   

  max_matches na_matches multi_matches regular  
2       [350]   [350 lb]            []      []

In [86]:
max_matches=df_max
max_matches[curation_col]=max_matches['max_matches']

In [87]:
max_matches[curation_col]=max_matches[curation_col].str.get(0)
max_matches[curation_col]=max_matches[curation_col].apply(lambda x: x+ ' lb')

In [88]:
max_matches[0:500]

product_id external_id  \
2      988798592    62739252   
3      817186606    62160568   
5     1237508673    60818188   
14    1340348872    69761307   
15    1339037348    69752483   
16    1239691926    69726356   
19    1143290731    69698323   
22    1336467368    69658770   
24    1158257488    69622374   
25    1261233310    69614394   
28    1127031299    69505582   
33    1336438671    69474677   
34     817405363    69475871   
40     988685796    61713789   
42     817430322    65844946   
44     988389849    65809556   
46     817314819    65476369   
49    1345417352    64961323   
51    1014120658    64689319   
52    1345405007    63830026   
55     817538645    62229050   
56    1345395254    67312542   
57    1345392894    41476727   
58     817267567    62704618   
62     817260169    61772250   
63     988600939    60715913   
64     817246884    60472717   
66     817241874    47422261   
67     817513991    47420236   
72     988588995    46333841   
76     817220930    43098996   
78    1345545738    42448198   
79    1345358224    42237006   
86     817482111    40646053   
95     817465855    46645692   
97    1327018287    69746302   
101   1327018192    69742453   
110   1338852695    69740961   
112   1338852569    69740809   
119   1015964477    69644167   
125   1224057359    69695472   
129    988566034    69629362   
144    817411761    69495540   
155    988562766    69554455   
156    817400711    69323010   
171   1203344992    69689151   
174    817414347    69506426   
179    817373977    68064983   
180    817373968    68064860   
183    817596644    67612826   
184    817359849    67602681   
188    817347929    67030217   
189    817369858    68147426   
190    817335376    66471929   
193    817333824    66382423   
204    817321970    65842836   
208   1017942063    69615333   
210    817319376    65698020   
223    817325452    46360687   
240    817300599    64646084   
265    817164406    43220694   
270   1349058067    69761597   
271   1349058066    69761606   
272   1341415681    69761326   
273   1341415523    69761328   
274   1330441102    69751383   
275   1338798954    69761228   
276   1337490088    69760862   
277   1349058072    69761601   
281   1349293858    69704670   
282   1207678049    69699522   
283   1330423375    69690981   
287   1330397381    69642757   
289    988490201    69632540   
307    817680177    69518561   
320    817369740    68130527   
325    817593580    67405817   
327    817675408    67044986   
328    817675960    67190829   
329    817589232    67104918   
330   1165668618    66585763   
331    988403302    66936473   
335    817674805    67044931   
337    817673577    66439417   
345   1328135246    65205556   
347    817279671    63372557   
348   1329419429    63087864   
349    817267960    62785464   
350   1338657318    77446528   
351   1333880109    60289438   
358    817225834    44591175   
362    817496651    41884461   
363    817217119    41884454   
378   1330678673    69747436   
381   1349058074    69761603   
402   1347986256    69761290   
403   1336478432    69707605   
404   1108562105    69687911   
409    817403377    69376498   
411    988719066    68744137   
413    817538637    62228763   
414   1060015610    60367716   
421    817475186    62077309   
425   1327018291    69746305   
426   1327018286    69746307   
433   1276376464    69685099   
436   1073896372    69679113   
437   1317921575    69686171   
453    817404104    69382864   
454   1144336390    69229282   
456    994509696    69327391   
463    817408702    69484597   
464    817625382    69338212   
469    817359847    67602728   
478    817321969    65842829   
513    817674842    67044535   
516    817675055    67039647   
521    817499559    43235193   
522    817221280    43235216   
523    817217120    41884478   
524   1347814744    44450939   
525    817671717    64719887   
527   1347743150    69761436   
529   1338688555    66176190   
530   

In [98]:
# df_max['max_matches']= df_max['max_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# df_max.head(1)

In [90]:
err

NameError: name 'err' is not defined

# matches

In [91]:
match_max=df_max

In [92]:
#import regex as re
matches=x
matches[curation_col]=matches['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: x+' lb')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches.head(3)

NameError: name 'x' is not defined

In [93]:
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\.",'',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'6393','63.93',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'Lbs','lb',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'lbs','lb',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'2645','264.5',str(measure)))

In [94]:
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'lb',' lb',str(measure)))

In [95]:
# match_max[curation_col]=df_max['max_matches']

In [96]:
# match_max[curation_col]=match_max['max_matches'].astype(str)

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda x: f'"{x}"')

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'881','881lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'\.','',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'330','330lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'Lb','lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'Lbs','lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'275','275lb',str(measure)))

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'825','82.5',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'2645','264.5',str(measure)))

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'lb',' lb',str(measure)))

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'lbs',' lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'lb lb',' lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'300 lb","300 lbs','300 lb',str(measure)))

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'300 lb","300  lb','300 lb',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'  ',' ',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'"113"','"113 lb"',str(measure)))
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'lb lb','lb',str(measure)))

In [ ]:
# match_max[curation_col]=match_max[curation_col].apply(lambda measure: re.sub(r'LB',' lb',str(measure)))

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [123]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/weight- {attribute}-max weight - match upload_Shawn.csv',index=False)

In [124]:
looks_good('Bed Bath & Beyond', attribute, df, max_matches)

In [26]:
def looks_good_reg(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/weight- {attribute}-weight-regular - match upload_Shawn.csv',index=False)

In [27]:
looks_good_reg('Bed Bath & Beyond', attribute, df, df_regular)

In [15]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/weight-NA {attribute}-weight - match upload_Shawn.csv',index=False)

In [16]:
looks_good_na('Bed Bath & Beyond', attribute, df, match_na)